# 1차모델 : ARIMA 
## seasonal = True 라서, sarima 모델이 더 정확한 표현인 것 같아요
## 시계열모델중 가장 기본적인 모델이라고 생각해서 사용했고
## 제품마다 AIC가 가장 작은 파리미터를 도출해서 엑셀에 따로 저장하고
## 최종 실행 셀만 돌려서 결과값 도출하였습니다
## 2주정도에 한번씩 파라미터 코드를 돌려서 엑셀 값들을 업데이트 해줬습니다
## 데이터는 토요일 기준으로 짤라서 직전 주까지의 데이터를 가져오고
## 모델을 돌리는 시점의 다음주 출고량을 예측하게됩니다

# 파라미터 도출 위한 코드
### EX 제품 : 삶은소건

In [2]:
import numpy as np
import pandas as pd
import pymssql
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from pmdarima.arima import auto_arima
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import matplotlib
from datetime import datetime 
from datetime import timedelta

#### SQL SERVER 에서 데이터 가져오기 , 출고량만 가져왔습니다

In [4]:
connet = pymssql.connect(server = '121.134.7.213',user='sa',password='twinet3015!', database='SEIERP_IYN_BAK')

sql_query = '''
declare	 @CD_CORP         NVARCHAR(2)
  , @DT_DEL_F        NVARCHAR(8)
  , @DT_DEL_T        NVARCHAR(8)

set @CD_CORP	=	'01'
set	@DT_DEL_F	=	'20200101'
set @DT_DEL_T	=	(SELECT	CONVERT(CHAR(8), GETDATE(), 112))

    SET NOCOUNT ON
	SET ANSI_WARNINGS OFF
	SET ARITHIGNORE ON
	SET ARITHABORT OFF

;With DeliveryCross AS (
 select a.CD_EMP
 , dbo.GetFormat_Day(B.DT_COMP,'-') as DT_COMP
 , B.NO_SMDL, B.NO_SMDLSUB
 , B.QT_SMDL, B.UP_PRICE, B.AM_PRICE, B.AM_VAT, B.AM_TOT, B.TX_NOTE, B.NO_SMOR
 , c.NM_CUST as NM_DELCUST
 , k.NM_USER, r.NM_WAREHOUSE
 , i.CD_ITEM, i.NM_ITEM, i.TX_SPEC, n.NM_TYPEDTL as NM_UNIT
 FROM SMDL_DELIVERY_H a
 Left outer Join SMDL_DELIVERY_D as b on a.CD_CORP=b.CD_CORP and a.NO_SMDL=b.NO_SMDL
 Left outer Join BSCT_CUST c on b.CD_CORP=c.CD_CORP and b.CD_DELCUST=c.CD_CUST
 Left outer Join BSIN_USER k on a.CD_CORP=k.CD_CORP and a.CD_EMP=k.CD_EMP
 Left outer Join STIT_WAREHOUSE r on b.CD_CORP=r.CD_CORP and b.CD_WAREHOUSE=r.CD_WAREHOUSE		--창고코드
 Left outer Join BSIT_ITEM i on b.CD_CORP=i.CD_CORP and b.CD_SYSITEM=i.CD_SYSITEM				--	품목코드
 Left outer Join (select * from BSIN_CODE_DTL) n on i.CD_CORP=n.CD_CORP and i.CD_UNIT=n.CD_TYPEDTL And n.CD_TYPE='MI001'
 Left outer Join (Select * From BSIN_CODE_DTL) t1 on b.CD_CORP=t1.CD_CORP and b.CD_SELLTYPE=t1.CD_TYPEDTL and t1.CD_TYPE='SM004'
 Left outer Join FROP_STORE m On c.CD_CORP = m.CD_CORP AND c.CD_STORECODE = m.CD_STORE			--매장코드
 WHERE 1=1
       AND A.CD_CORP    = @CD_CORP
	   AND a.DT_COMP	>=	@DT_DEL_F 
	   AND a.DT_COMP	<=	@DT_DEL_T
       AND ISNULL(B.CK_BUY, 0) <> 9			--공장출고만
 )
 SELECT 1 div_out, DT_COMP, CD_ITEM, NM_ITEM , TX_SPEC , NM_UNIT
 , sum(QT_SMDL) QT_SMDL, ((sum(AM_PRICE)) / sum(QT_SMDL)) UP_PRICE --avg(UP_PRICE) UP_PRICE
 , sum(AM_VAT) AM_VAT, sum(AM_PRICE) AM_PRICE, sum(AM_PRICE)+sum(AM_VAT) AM_TOT
 FROM DeliveryCross
 GROUP BY CD_ITEM, NM_ITEM  , TX_SPEC , NM_UNIT, DT_COMP
 UNION ALL
 SELECT 2 div_out, '', '총계' itemcode1, '' itemname , '' ,''
 , sum(QT_SMDL) QT_SMDL, ((sum(AM_PRICE)) / sum(QT_SMDL)) UP_PRICE  --, avg(UP_PRICE) UP_PRICE
 , sum(AM_VAT) AM_VAT, sum(AM_PRICE) AM_PRICE, sum(AM_PRICE)+sum(AM_VAT) AM_TOT
 FROM DeliveryCross
 order by div_out, CD_ITEM
 '''

row_data = pd.read_sql(sql=sql_query,con=connet)

df1 = row_data[row_data['NM_ITEM']=='삶은소건(2kg)'][['DT_COMP','QT_SMDL']]
df1.columns = ['ds','y']
df1['ds'] = pd.to_datetime(df1['ds'])
df1.set_index(df1['ds'],inplace = True)
df2 = df1.resample('D').sum()
df3 = df2.reset_index()
while (df3['ds'].iloc[len(df3)-1].weekday() != 5 ):
    df3 = df3.drop(len(df3)-1,axis=0)
    df3['index'] = df3['ds']
df3.set_index(df3['index'],inplace = True)
df4 = df3.drop(['index'],axis=1)
week_df = df4.resample('W-Sat').sum()
week_df['ds'] = week_df.index
data = week_df.reset_index()
forecast_data = data.drop(['index'],axis=1)  
data = forecast_data.set_index('ds')
data

C:\Users\ioi\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,y
ds,
2020-01-04,70.0
2020-01-11,80.0
2020-01-18,98.0
2020-01-25,116.0
2020-02-01,70.0
...,...
2023-01-28,72.0
2023-02-04,72.0
2023-02-11,110.0


#### 데이터셋 shuffle = False 로해서 시간순대로 데이터 고정
#### auto_arima는 가장 AIC가 낮은 파라미터 조합을 찾아주는 함수
#### 각 파라미터의 범위를 설정해줌 , seasonal = True로 계절성 있다고 설정

In [5]:
from sklearn.model_selection import train_test_split
train_data , test_data = train_test_split(data , test_size=0.1,shuffle = False)

s_auto_arima_model = auto_arima(train_data , start_p = 1 , start_q = 1,
                             max_p = 3 , max_q = 3  , seasonal = True, m=12 , D=2,
                             d=1,  max_P = 3 , max_Q = 3,
                             trace = True,
                             error_action = 'ignore',
                             suppress_warnings = True , stepwise = False)

s_auto_arima_model.summary()

 ARIMA(0,1,0)(0,2,0)[12]             : AIC=1444.078, Time=0.20 sec
 ARIMA(0,1,0)(0,2,1)[12]             : AIC=inf, Time=0.33 sec
 ARIMA(0,1,0)(0,2,2)[12]             : AIC=inf, Time=1.74 sec
 ARIMA(0,1,0)(0,2,3)[12]             : AIC=inf, Time=4.78 sec
 ARIMA(0,1,0)(1,2,0)[12]             : AIC=1384.502, Time=0.14 sec
 ARIMA(0,1,0)(1,2,1)[12]             : AIC=inf, Time=1.03 sec
 ARIMA(0,1,0)(1,2,2)[12]             : AIC=inf, Time=2.89 sec
 ARIMA(0,1,0)(1,2,3)[12]             : AIC=inf, Time=3.96 sec
 ARIMA(0,1,0)(2,2,0)[12]             : AIC=1354.013, Time=0.46 sec
 ARIMA(0,1,0)(2,2,1)[12]             : AIC=inf, Time=1.20 sec
 ARIMA(0,1,0)(2,2,2)[12]             : AIC=inf, Time=3.53 sec
 ARIMA(0,1,0)(2,2,3)[12]             : AIC=inf, Time=5.52 sec
 ARIMA(0,1,0)(3,2,0)[12]             : AIC=1328.888, Time=1.06 sec
 ARIMA(0,1,0)(3,2,1)[12]             : AIC=inf, Time=4.88 sec
 ARIMA(0,1,0)(3,2,2)[12]             : AIC=inf, Time=6.14 sec
 ARIMA(0,1,1)(0,2,0)[12]             : AIC=1384.15

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  147
Model:             SARIMAX(0, 1, 1)x(3, 2, [], 12)   Log Likelihood                -633.498
Date:                             Fri, 24 Feb 2023   AIC                           1276.997
Time:                                     15:26:29   BIC                           1291.017
Sample:                                          0   HQIC                          1282.692
                                             - 147                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1         -0.7480      0.045    -16.592      0.000      -0.836      -0.660
ar.S.L12      -1.2030      0.073    -16.472      0.000      -1.346      -1.060
ar.S.L24      -0.9691      0.133     -7.283      0.000      -1.230      -0.708
ar.S.L36      -0.4923      0.147     -3.358      0.001      -0.780      -0.205
sigma2      1559.6322    166.262      9.381      0.000    1233.765    1885.499
===================================================================================
Ljung-Box (L1) (Q):                   0.02   Jarque-Bera (JB):               173.62
Prob(Q):                              0.90   Prob(JB):                         0.00
Heteroskedasticity (H):               7.63   Skew:                            -0.89
Prob(H) (two-sided):                  0.00   Kurtosis:                         8.57
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## 제품별로 최적 파라미터 넣은 엑셀 파일 작성

In [8]:
df_para = pd.read_excel('C:\\Users\\ioi\\Desktop\\시계열\\ARIMA 모델\\arima_파라미터.xlsx')
df_para

,품명,모델,order,seasonal_order
0,한촌탕고기(뒷덩이),SARIMAX,"1,1,1","1,2,2,12"
1,한촌수육양지,SARIMAX,"1,1,1","3,2,0,12"
2,한촌설렁탕육수(2.5kg),SARIMAX,"3,1,0","0,2,2,12"
3,삶은소건(2kg),SARIMAX,"0,1,1","3,2,0,12"
4,사골추출액-1202(냉장),SARIMAX,"0,1,1","3,2,0,12"
5,한촌매운돼지갈비소스,SARIMAX,"1,1,2","0,2,2,12"
6,한촌양념불고기,SARIMAX,"3,1,1","0,2,1,12"
7,곰탕육수(구.나주곰탕육수),SARIMAX,"1,1,1","0,2,2,12"
8,혼합김치 18Kg,SARIMAX,"0,1,1","3,2,0,12"


## 모델 실행 코드

In [9]:
item_name = df_para['품명'].to_list()

result = pd.DataFrame(columns= ['품명','품목코드','날짜','수량'])

for i in range(len(item_name)) :
    df1 = row_data[row_data['NM_ITEM']==item_name[i]][['DT_COMP','QT_SMDL']]
    df1.columns = ['ds','y']
    df1['ds'] = pd.to_datetime(df1['ds'])
    df1.set_index(df1['ds'],inplace = True)
    df2 = df1.resample('D').sum()
    df3 = df2.reset_index()
    while (df3['ds'].iloc[len(df3)-1].weekday() != 5 ):
        df3 = df3.drop(len(df3)-1,axis=0)

    df3['index'] = df3['ds']
    df3.set_index(df3['index'],inplace = True)
    df4 = df3.drop(['index'],axis=1)

    week_df = df4.resample('W-Sat').sum()
    week_df['ds'] = week_df.index
    data = week_df.reset_index()
    forecast_data = data.drop(['index'],axis=1)  
    data = forecast_data.set_index('ds')
    
    order = df_para[df_para['품명']==item_name[i]]['order'].values.sum()
    order1 = []
    for j in order.split(','):
        order1.append(int(j))
    real_order = tuple(order1)
    
    s_order = df_para[df_para['품명']==item_name[i]]['seasonal_order'].values.sum()
    s_order1 = []
    for j in s_order.split(','):
        s_order1.append(int(j))
    real_s_order = tuple(s_order1)
    
    m = SARIMAX(data,order = real_order , seasonal_order = real_s_order)
    pre = m.fit().forecast(2)
    
    result.loc[len(result)] = [item_name[i],
                               row_data[row_data['NM_ITEM']==item_name[i]]['CD_ITEM'].iloc[0],
                                forecast_data[-1:]['ds'].iloc[0] + timedelta(weeks=2),
                               round(pre[1])]
    
result.to_excel('C:\\Users\\ioi\\Desktop\\시계열\\ARIMA 모델\\{0}(토기준)_출고예측결과.xlsx'.format(str(forecast_data[-1:]['ds'].iloc[0] + timedelta(weeks=2)).split(' ')[0]), index=False)
result

C:\Users\ioi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency W-SAT will be used.
  self._init_dates(dates, freq)
C:\Users\ioi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency W-SAT will be used.
  self._init_dates(dates, freq)
C:\Users\ioi\anaconda3\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\ioi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency W-SAT will be used.
  self._init_dates(dates, freq)
C:\Users\ioi\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency W

,품명,품목코드,날짜,수량
0,한촌탕고기(뒷덩이),A01010303,2023-03-04,96
1,한촌수육양지,A01010201,2023-03-04,159
2,한촌설렁탕육수(2.5kg),A01030104,2023-03-04,8597
3,삶은소건(2kg),A01010404,2023-03-04,96
4,사골추출액-1202(냉장),A01030203,2023-03-04,1000
5,한촌매운돼지갈비소스,A01080101,2023-03-04,90
6,한촌양념불고기,A01020301,2023-03-04,521
7,곰탕육수(구.나주곰탕육수),A01031101,2023-03-04,150
8,혼합김치 18Kg,G01010101,2023-03-04,959
